https://raghakot.github.io/keras-vis/visualizations/saliency/

In [ ]:
import numpy as np
import keras.backend as K
from keras.models import load_model
import os
import pandas as pd
import matplotlib.pyplot as plt


Using TensorFlow backend.


In [ ]:
model = load_model('my_model.h5')
model.summary()

In [ ]:
x_train = os.path.join(os.getcwd(),'data','x_train.npy')
x_train = np.load(x_train)

In [ ]:
def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    # [3,4] after normalize => [3/5,4/5], [3/12.5^2, 4/12.5^2]

    # K.epsilon()) = 10^-7, prevent from dividing by zero
    
    return x / ( K.sqrt(K.mean(K.square(x))) + K.epsilon() )


In [ ]:

index_image = 22
input_img_data = x_train[index_image].reshape(1,48,48,1)
predict_vector = model.predict(input_img_data)
index_category = np.argmax(predict_vector)

input_img = model.input

layer_output = model.layers[-1].output
# loss = K.mean(layer_output[:,index_category])
loss = layer_output[:,index_category]

grads = K.gradients(loss, [input_img])[0]
grads = normalize(grads)

fn = K.function([input_img], [loss,grads,layer_output])


loss_value, grads_value, layer_output = fn([input_img_data])

print('index_category',index_category)
print('loss_value',loss_value)
print('grads_value',grads_value)
print('layer_output',layer_output)

# print(grads_value.shape)
plt.imshow(x_train[index_image,:,:,0])
plt.show()
plt.imshow(grads_value[0,:,:,0], cmap = 'jet')
plt.show()
